In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import std, mean, sqrt
from scipy.spatial.distance import directed_hausdorff
%matplotlib inline

In [ ]:
def overlap_function(base_arr, input_arr, input_ids_list, input_mode = 'separate',\
    base_ids_list = None, basis = 'DICE'):
    '''
    Measures the overlap between the base and values of the the input.
    The values of the input to be measured against are provided in the input_ids_list.
    Measures are based on 'DICE', 'base', or 'input'.
    
    Parameters:
        base_arr: arr
            The base array.
            
        input_arr: arr
            The input array.
            
        input_ids_list: list
            The list of IDs in the input to be considered.
            
        input_mode: {'separate', 'bundle'}
            If separate, all values in input_ids_list are measured separately.
            If bundle, all values in input_ids_list are considered as one ROI.
            
        base_ids_list: list, default = None
            The list of IDs in the base to be considered as one bundle.
            If None, all non-zero values are considered.
            
        basis: {'DICE', 'base', 'input'}
            The basis of overlap measure.
    
    Returns:
        overlap_list: list
            The list of overlap ratios.
    '''

    # Making an array of zeros with the same size
    masked_base = np.zeros_like(base_arr)
    masked_input = np.zeros_like(input_arr)

    # Masking the base based on base_ids_list
    if base_ids_list != None:
        for ID in base_ids_list:
            masked_base += np.where(base_arr == ID, 1, 0)
    
    # Converting base vector to a binary vector, in case it is not
    elif base_ids_list == None:
        masked_base = np.where(base_arr != 0, 1, 0)
    
    # Getting count of considered voxels in base
    count_base = np.sum(masked_base)

    # Creating lists to store overlap counts
    count_input_list = []
    count_overlap_list = []
    
    # Creating a list to store final overlap ratios
    overlap_list = []
    
    # Masking the input in 'bundle' mode
    if input_mode == 'bundle':
        for ID in input_ids_list:
            masked_input += np.where(input_arr == ID, 1, 0)

        # Counting the input and adding to the list
        count_input = np.sum(masked_input)
        count_input_list.append(count_input)

        # Counting the overlap and adding to the list
        count_overlap = np.vdot(masked_base, masked_input)
        count_overlap_list.append(count_overlap)

    # Measuring overlap for 'separate' mode
    elif input_mode == 'separate':
        for ID in input_ids_list:
            # Masking input for that specific ID
            masked_input = np.where(input_arr == ID, 1, 0)
            
            # Count of voxels in the ROI and adding to the list
            count_input = np.sum(masked_input)
            count_input_list.append(count_input)

            # Counting the overlap and adding to the list
            count_overlap = np.vdot(masked_base, masked_input)
            count_overlap_list.append(count_overlap)
    
    # Computing on different bases
    if basis == 'DICE':
        # Total count of base ROI and input ROI
        total_count_list = [x + count_base for x in count_input_list]
    
        # Overlap list
        overlap_list = [(2*x)/y for x, y in zip(count_overlap_list, total_count_list)]
    
    elif basis == 'base':
        overlap_list = [(x/count_base) for x in count_input_list]
    
    elif basis == 'input':
        overlap_list = [x/y for x, y in zip(count_overlap_list, count_input_list)]
    
    return overlap_list

In [ ]:
def DICE_length_function(region_ids_list, cluster_vec):
    '''
    Plotting DICE score for brain regions over cluster numbers.
    
    Args:        
        region_ids_list: list
            List of brain region IDs to visualize.
        
        cluster_vec: vec
            The clusters vector.
    
    Returns:
        plot
    '''
    
    # Making lists to save DICE scores and length rations
    DICE_list = []
    length_list = []
    
    # Getting the number of clusters
    n_clusters = np.unique(cluster_vec).shape[0]
    cluster_ids = list(range(n_clusters))
    
    # Loading the anatomy file
    ant_file = '/data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/200um/allen_annot200.nii'
    ant_vec = nifti_to_array(ant_file)
    
    for count, ID in enumerate(region_ids_list):
        # Selecting a brain region
        region_ID = [ID]
        
        # Computing the overlap for the cluster
        overlap_list = overlap_function(base_arr = ant_vec, input_arr = cluster_vec,\
        base_ids_list = region_ID, input_ids_list = cluster_ids)

        # Adding the maximum DICE score to the list
        top_DICE = max(overlap_list)
        DICE_list.append(top_DICE)
        
        # Computing the length
        length = region_length(ID)
        length_list.append(length)
        
    return length_list, DICE_list

In [ ]:
def DICE_orientation_function(region_ids_list, cluster_vec):
    '''
    Plotting DICE score for brain regions over cluster numbers.
    
    Args:        
        region_ids_list: list
            List of brain region IDs to visualize.
        
        cluster_vec: vec
            The clusters vector.
    
    Returns:
        plot
    '''
    
    # Making lists to save DICE scores and length rations
    DICE_list = []
    length_list = []
    
    # Getting the number of clusters
    n_clusters = np.unique(cluster_vec).shape[0]
    cluster_ids = list(range(n_clusters))
    
    # Loading the anatomy file
    ant_file = '/data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/200um/allen_annot200.nii'
    ant_vec = nifti_to_array(ant_file)
    
    for count, ID in enumerate(region_ids_list):
        # Selecting a brain region
        region_ID = [ID]
        
        # Computing the overlap for the cluster
        overlap_list = overlap_function(base_arr = ant_vec, input_arr = cluster_vec,\
        base_ids_list = region_ID, input_ids_list = cluster_ids)

        # Adding the maximum DICE score to the list
        top_DICE = max(overlap_list)
        DICE_list.append(top_DICE)
        
        # Computing the length
        length = find_orientation(ID)
        length_list.append(length)
        
    return length_list, DICE_list

In [ ]:
def copy_nifti_header(base, input, outfile):
    """
    Takes the NIFTI header from base file and overwrites it for the input
    (Nibabel library needed)
    
    Parameters:
        base (str): base file address, its header gets copied to input's
        input (str): input file address, its header gets replaced with base's
        outfile (str): output file address
    
    Returns:
        None
    """
    
    base_img = nib.load(base)
    
    input_img = nib.load(input)
    input_matrix = np.array(input_img.dataobj).astype('float32')
    
    copy_img = nib.Nifti1Image(input_matrix, header=base_img.header, affine=base_img.affine)
    nib.save(copy_img, outfile)

In [ ]:
def array_to_nifti(arr, output_file, master_file = None):
    """
    Takes an array and saves it as NIFTI file in the given output address
    
    Parameters:
        arr (array): matrix of image voxels.
        output_file (string): output file path.
        master_file (str): Master file to copy header from.
    
    Returns:
        None
    """
    
    # Saving the NII with the same header as master file
    if master_file != None:
        master_img = nib.load(master_file)
        output_img = nib.Nifti1Image(arr, header = master_img.header, affine = master_img.affine)
        nib.save(output_img, output_file)
    
    # Saving NII file with a general header
    else:
        save = nib.Nifti1Image(arr, np.eye(4))
        save.header.get_xyzt_units()
        save.to_filename(os.path.join('build',output_file))

In [ ]:
def nifti_to_array(nii_file):
    '''
    Takes an input file path (str) to the NIFTI file and returns the 3D array.
    
    Parameters:
        nii_file (str): The address to nii file
    
    Returns:
        vector (np.array): The flattened 1D array

    '''
    img = nib.load(nii_file)
    array = np.array(img.dataobj)
    
    return array

In [ ]:
def nifti_to_vector(nii_file):
    '''
    Takes an input file address (str) to the NIFTI file and returns
    the flattened vector of the input array.
    
    Parameters:
        nii_file (str): The address to nii file
    
    Returns:
        vector (np.array): The flattened 1D array
    '''
    img = nib.load(nii_file)
    array = np.array(img.dataobj)
    vector = array.flatten()
    
    return vector

In [ ]:
def stack_nii(path_prefix, n):
    '''
    Takes the path to NIFTI files and stacks them in rows.
    
    Parameters:
        path_prefix (str): Prefix of the path to the NIFTI file until the number,
            example: '/data/img' if the complete location is: '/data/img1.nii'
    
        n (int): Number of NIFTI files to input
    
    Returns:
        stack (array): Numpy array with n rows
    '''
    input_list = [path_prefix+str(i)+'.nii' for i in range(1,n+1)]
    stack = np.vstack([nifti_to_vector(input)[0]] for input in input_list)
    
    return stack

In [ ]:
def cohend_func(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    d = (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    return d

In [ ]:
def cohen_d(stack1, stack2):
    '''
    Takes 2 arrays and computes then Cohen's d using the cohend_func function
    
    Parameters:
        stack1, stack2 (array): Arrays with subjects as rows and voxels as columns
    
    Returns:
        d (vector): Vector with size of stack1/stack2 columns
    '''
    d = np.zeros(stack1.shape[1])
    for i in range (stack1.shape[1]):
        d[i] = cohend_func(stack1[:,i], stack2[:,i])
        
    return d

In [ ]:
def std_vector(vector):
    '''
    Standardizes the given vector.
    
    Arguments:
        vector(vec): Input vector
    
    Returns:
        std_vector (vec): The standardized vector
    '''
    
    # The maximum value in the vector
    max_vector = np.amax(vector)
    
    # Dividing by the maximum value to standardize the vector
    std_vector = vector/max_vector
    
    return std_vector

In [ ]:
def nifti_to_npy(nifti_file, output_file):
    '''
    Takes the array of an NII file and saves as NPY file.
    
    Arguments:
        nifti_file(str): The input NII path.
        output_file(str): The output NPY file.
        
    Returns:
        None.
    '''
    arr = nifti_to_array(nifti_file)
    np.save(output_file, arr)

In [ ]:
def reconstruct_ABA (vector, indices_file, outside_value = -1, mirror = True, array_3D = True):
    '''
    This function reconstructs the masked and/or halved
    vector to the original shape (159326,).
    
    Args:
        vector: vec
            The masked vector.
        
        indices_file: str
            The path to the indices file.
        
        outside_value: int, default = -1
            The value set for outside of brain voxels.
        
        scale = bool, default = False
            If True, the values get scaled up to avoid becoming zero.
            Used for features and values between 0 and 1.
        
        mirror: bool, default = True
            If True, the vector gets mirrored (such as half-brain cases).
        
        array_3D: bool, default = True
            if True, an array of size (67,58,41) is returned.
    
    Returns:
        output_3D: array
            output vector/array
    '''
    # Loading the indices
    indices = np.load(indices_file)
    
    # Making empty array to reconstruct the original array
    output = np.full((159326), outside_value, dtype='float32')
    
    # Reconstructing the array
    output[indices] = vector

    # Making a 3D output
    output_3D = output.reshape(67,58,41)
    
    # Mirroring the array
    if mirror == True:
        for i in range(29):
            output_3D[:,57-i,:] = output_3D[:,i,:]
    
    # If 3D array is not requested
    if array_3D == False:
        output_3D = output.flatten()
    
    return output_3D

In [ ]:
def plot_regions_DICE(region_ids_list, cluster_path):
    '''
    Plotting DICE score for brain regions over cluster numbers.
    
    Args:        
        region_ids_list: list
            List of brain region IDs to visualize.
        
        cluster_path: str
            The path to the cluster file.
    
    Returns:
        plot
    '''
    
    # List of K values
    number_range = list(range(1,50))
    ext_range = list(range(50,551,50))
    number_range.extend(ext_range)
    last_number = 594
    number_range.append(last_number)

    # Number of values in the range
    len_range = len(number_range)

    # Loading the anatomy file
    ant_file = '/data/bioprotean/RAG2/AVG/MWT_avg/to_allen/overlap/200um/allen_annot200.nii'
    ant_vec = nifti_to_array(ant_file)
    
    # Making a dictionary to save scores
    score_dict = {}

    for count, ID in enumerate(region_ids_list):
        # Selecting a brain region
        region_ID = [ID]

        # Creating an array to store overlap ratios
        score_arr = np.zeros((last_number, len_range))

        # Looping over different clusters
        for i, n_clusters in enumerate(number_range):
            # Inputting path to the cluster file
            cluster_file = cluster_path+str(n_clusters)+'_clusters.npy'
            cluster_vec = np.load(cluster_file)
            cluster_ids = list(range(n_clusters))

            # Computing the overlap for the cluster
            overlap_list = overlap_function(base_arr = ant_vec, input_arr = cluster_vec,\
            base_ids_list = region_ID, input_ids_list = cluster_ids)

            # Adding the overlap list to the array
            score_arr[:n_clusters, i] = np.array(overlap_list)

        # Adding the array to the dictionary
        score_dict[count] = score_arr
    
    # Setting the figure size
    plt.rcParams["figure.figsize"] = (7,5)

    # Initiating the plots
    fig, ax = plt.subplots()

    # Iterating over dictionary items
    for i, (j, score_arr) in enumerate(score_dict.items()):

        # List of max overlap from each column
        max_ratio_list = []

        for z in range(len_range):
            max_ratio = np.amax(score_arr[:,z])
            max_ratio_list.append(max_ratio)

        # Defining x and y
        x = number_range
        y = max_ratio_list

        # print('Maximum ratio is {} for {} clusters.\n'.format(max(y), y.index(max(y))+1))

        # Plotting the ratios
        ax.plot(x, y)

    # Setting the grid style
    sns.set(style = "darkgrid")

    # Manual set of grid values and labels
    plt.xscale('log')
    x_values = [1,10,25,50,100,200,300,400,500,594]
    ax.set_xticks(x_values)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 14)

    # Naming the x-axis, y-axis and the whole graph
    plt.xlabel("Number of K-means clusters", fontsize = 16)
    plt.ylabel("DICE coefficient", fontsize = 16)
    plt.title("Maximum DICE coefficient", fontsize = 16)
    plt.legend(['RegionID_'+str(ID) for ID in region_ids_list], fontsize = 10)

    # Visualizing
    plt.show()

In [ ]:
def find_orientation(region_id):
    '''
    Showing the main orientation of the region.
    
    Args:
        region_id: int
            ID of the brain region.
    
    Returns:
        orientation: int
            Main orientation of the region.
    '''
    
    # Loading the Allen Reference Atlas
    allen_path = '/data/bioprotean/ABA/PCA/80_variance/allen_annot200.nii'
    reference = nifti_to_array(allen_path)
    
    # Masking the ARA for the brain region
    mask_region = np.where(reference == region_id)
    
    # Lengths in different locations
    length_0 = mask_region[0][-1] - mask_region[0][0]
    length_1 = mask_region[1][-1] - mask_region[1][0]
    length_2 = mask_region[2][-1] - mask_region[2][0]
    
    # Choosing the orientation
    length_list = [length_0, length_1, length_2]
    max_value = max(length_list)
    orientation = length_list.index(max_value)
    
    return orientation

In [ ]:
def region_length(region_id):
    '''
    Showing the main orientation of the region.
    
    Args:
        region_id: int
            ID of the brain region.
    
    Returns:
        ratio: float
            Ratio of Sagittal/Axial length.
    '''
    
    # Loading the Allen Reference Atlas
    allen_path = '/data/bioprotean/ABA/PCA/80_variance/allen_annot200.nii'
    reference = nifti_to_array(allen_path)
    
    # Masking the ARA for the brain region
    mask_region = np.where(reference == region_id)
    
    # Lengths in different locations
    length_0 = mask_region[0][-1] - mask_region[0][0]
    length_1 = mask_region[1][-1] - mask_region[1][0]
    length_2 = mask_region[2][-1] - mask_region[2][0]
    
    # Computing the ratio
    ratio = length_0 / length_1
    
    return ratio